### A diagram using cytoscape

The following exercise demonstrates how to create
a diagram using `cytoscape.js` with `js_proxy` making
use of complex styling features.

The diagram is combined with other ipywidget controls
to provide additional interactivity

In [ ]:
# allow import without install
import sys
if ".." not in sys.path:
    sys.path.append("..")

from jp_gene_viz import js_proxy
from jp_gene_viz import js_context

import ipywidgets as widgets
from IPython.display import display

In [ ]:
js_context.load_if_not_loaded(["cytoscape.js"])
js_proxy.load_javascript_support()

In [ ]:
# cytoscape styling for nodes and edges and parent nodes.
style = [
    {
      "selector": 'node',
      "css": {
        'content': 'data(label)',
        'text-valign': 'center',
        'text-halign': 'center',
        'text-wrap': 'wrap',
        'text-max-width': 80,
        'text-valign': 'center',
        'color': 'white',
        'text-outline-width': 2,
        'text-outline-color': '#888',
            "background-color": "cyan"
      }
    },
    {
      "selector": '$node > node',
      "css": {
        'padding-top': '10px',
        'padding-left': '10px',
        'padding-bottom': '10px',
        'padding-right': '10px',
        'text-valign': 'top',
        'text-halign': 'left',
        'background-color': '#bbb'
      }
    },
    {
      "selector": 'edge',
      "css": {
        'target-arrow-shape': 'triangle',
        "line-color": "red",
        'target-arrow-color': 'red',
      }
    },
    {
      "selector": ':selected',
      "css": {
        'background-color': 'black',
        'line-color': 'black',
        'target-arrow-color': 'black',
        'source-arrow-color': 'black'
      }
    }]

In [ ]:
def nodedesc(ident, label, parent=None, pos=None):
    data = { "id": ident, "label": label}
    if pos:
        (x, y) = pos
        position = {"x": x, "y": y}
    if parent:
        data["parent"] = parent
    result = {"data": data}
    if pos:
        result["position"] = position
    return result

def edgedesc(source, target):
    ident = source+target
    return {"data": {"id": ident, "source": source, "target": target}}

def makeNetwork(w, with_proxy=False, layout="preset"):
    #w = js_proxy.ProxyWidget()
    elt = w.element()
    w(elt.empty())
    window = w.window()
    cytoscape = window.cytoscape
    jQuery = window.jQuery
    w(elt.height(800).width(800))
    w(elt._set("target", 
               jQuery("<div></div>").
               height(800).
               width(800).
               #html("cytoscape target div").
               appendTo(elt)
              ))
    nodes = [{"data": {"id": "n", "label": "cytoscape"}}]
    #style= 'node { content: data(id); }'
    nodes = []
    edges = []
    nodes.append(nodedesc("ip", "IPython"))
    nodes.append(nodedesc("py", "Python widget"))
    nodes.append(nodedesc("jv", "Javacript view"))
    nodes.append(nodedesc("jp", "Javascript plugin"))
    if with_proxy:
        nodes.append(nodedesc("jspw", "js_proxy widget", "py", (400,300)))
        nodes.append(nodedesc("jspv", "js_proxy_view", "jv", (400, 450)))
        edges.append(edgedesc("jspw", "jspv"))
        edges.append(edgedesc("jspv", "jspw"))
    x = 0
    for component in ["picker", "network", "heatmap"]:
        x = x + 200
        letter = component[0]
        nc = letter + "c"
        nw = letter + "w"
        nv = letter + "v"
        np = letter + "p"
        nodes.append(nodedesc(nc, component+" config", "ip", (x, 150)))
        if not with_proxy:
            nodes.append(nodedesc(nw, component+" widget", "py", (x, 300)))
            nodes.append(nodedesc(nv, component+" view", "jv", (x, 450)))
        nodes.append(nodedesc(np, component+" plugin", "jp", (x, 600)))
        if not with_proxy:
            for (a,b) in [(nc, nw), (nw,nv), (nv,np)]:
                edges.append(edgedesc(a, b))
                edges.append(edgedesc(b, a))
        else:
            for (a,b) in [(nc, "jspw"), ("jspv", np)]:
                edges.append(edgedesc(a, b))
                edges.append(edgedesc(b, a))
    elements = {"nodes": nodes, "edges": edges}
    #elements = {
    #                "nodes":[{ "data": { "id": "n", "label": "cytoscape.js" }} ]
    #            }
    descriptor = {
        "container": elt.target._get(0),
        "style": style, 
        "elements": elements,
        "layout": {"name": layout, "padding": 5}
    }
    w(elt.width("200px").height("200px"))
    w(cytoscape(descriptor))
    w.flush()
    return (w, descriptor)

class Diagram(object):
    
    def __init__(self):
        w = self.widget = js_proxy.ProxyWidget()
        c = self.check = widgets.Checkbox(description="with js_proxy", value=False)
        c.on_trait_change(self.draw, "value")
        options = ["preset", "cose"]
        d = self.dropdown = widgets.Dropdown(options=options, value=options[0])
        d.on_trait_change(self.draw, "value")
        a = self.assembly = widgets.VBox(children=[c, d, w])
        # force the assembly to be big enough
        a.height = 950
        self.draw()
        display(self.assembly)
        
    def draw(self, *ignored_arguments):
        makeNetwork(self.widget, self.check.value, self.dropdown.value)

In [ ]:
# Old way:
#w = js_proxy.ProxyWidget()
#dummy = makeNetwork(w, True)
#display(w)

# New way:
D = Diagram()